# **Sentiment Analysis using spacy 3.0**

**1. Installing Libraries**

In [ ]:
# Installing Spacy library

!pip install spacy==3.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.5 MB 26.1 MB/s 
     |████████████████████████████████| 13.7 MB 70.1 MB/s 
     |████████████████████████████████| 671 kB 71.2 MB/s 
  Attempting uninstall: typer
    Found existing installation: typer 0.7.0
    Uninstalling typer-0.7.0:
      Successfully uninstalled typer-0.7.0
  Attempting uninstall: pydantic
    Found existing installation: pydantic 1.10.2
    Uninstalling pydantic-1.10.2:
      Successfully uninstalled pydantic-1.10.2
  Attempting uninstall: thinc
    Found existing installation: thinc 8.1.5
    Uninstalling thinc-8.1.5:
      Successfully uninstalled thinc-8.1.5
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.4
    Uninstalling spacy-3.4.4:
      Successfully uninstalled spacy-3.4.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 1.1 MB/s 
     |████████████████████████████████| 1.2 MB 59.8 MB/s 
     |████████████████████████████████| 5.8 MB 61.7 MB/s 
     |████████████████████████████████| 6.6 MB 75.7 MB/s 
     |████████████████████████████████| 827 kB 59.0 MB/s 
     |████████████████████████████████| 7.6 MB 58.7 MB/s 
     |████████████████████████████████| 182 kB 80.3 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 8.0.17
    Uninstalling thinc-8.0.17:
      Successfully uninstalled thinc-8.0.17
  Attempting uninstall: spacy
    Found existing installation: spacy 3.1.1
    Uninstalling spacy-3.1.1:
      Successfully uninstalled spacy-3.1.1


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 907 kB/s 
     |████████████████████████████████| 1.2 MB 56.1 MB/s 
     |████████████████████████████████| 5.8 MB 58.8 MB/s 
     |████████████████████████████████| 182 kB 71.5 MB/s 
     |████████████████████████████████| 7.6 MB 58.6 MB/s 


In [ ]:
# Downloading the spaCy Transformer model "en_core_web_trf"
!python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 460.3 MB 26 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


**2. Importing necessary library**

In [3]:
# Importing libraries

import pandas as pd
from datetime import datetime
import spacy
import spacy_transformers

# Storing docs in binary format
from spacy.tokens import DocBin

**3. Reading Dataset and Pre-processing**

In [ ]:
# Reading the dataset
df = pd.read_csv("/content/drive/MyDrive/TrueFoundry/airline_sentiment_analysis.csv", index_col=0)
# Removing tags, links, hashtags 
df['text'] = df['text'].str.replace('http\S+|www.\S+|@\S+|#\S+', '', case=False)
df.head()

<ipython-input-18-ea7d6fcaa3ef>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  df['text'] = df['text'].str.replace('http\S+|www.\S+|@\S+|#\S+', '', case=False)


,airline_sentiment,text
1,positive,plus you've added commercials to the experien...
3,negative,"it's really aggressive to blast obnoxious ""en..."
4,negative,and it's a really big bad thing about it
5,negative,seriously would pay $30 a flight for seats th...
6,positive,"yes, nearly every time I fly VX this “ear wor..."


In [ ]:
df['text'].iloc[0]

" plus you've added commercials to the experience... tacky."

In [ ]:
df['airline_sentiment'].value_counts()

negative    9178
positive    2363
Name: airline_sentiment, dtype: int64

In [ ]:
df_pos = df[df['airline_sentiment']=='positive']
df_updated = df[df['airline_sentiment']=='negative'].sample(df_pos.shape[0]).append(df_pos)

In [ ]:
df_updated['airline_sentiment'].value_counts()

negative    2363
positive    2363
Name: airline_sentiment, dtype: int64

**4. Splitting the processed dataset**

In [ ]:
#Splitting the dataset into train and test
train = df_updated.sample(frac = 0.8, random_state = 25)
test = df_updated.drop(train.index)

In [ ]:
# Checking the shape

print(train.shape, test.shape)

(3781, 2) (945, 2)


In [ ]:
test['airline_sentiment'].value_counts()

positive    480
negative    465
Name: airline_sentiment, dtype: int64

**5. Converting train and test dataset into binary format**

In [ ]:
import spacy
nlp=spacy.load("en_core_web_trf")

In [ ]:
#Creating tuples
train['tuples'] = train.apply(lambda row: (row['text'],row['airline_sentiment']), axis=1)
train = train['tuples'].tolist()
test['tuples'] = test.apply(lambda row: (row['text'],row['airline_sentiment']), axis=1)
test = test['tuples'].tolist()
train[0]

(' - Whoooo Hooooo just crossed 25,367!!! At my current pace I should reach 152,202 for the year.',
 'positive')

In [ ]:
# User function for converting the train and test dataset into spaCy document
def document(data):
  #Creating empty list called "text"
  text = []
  for doc, label in nlp.pipe(data, as_tuples = True):
    if (label=='positive'):
      doc.cats['positive'] = 1
      doc.cats['negative'] = 0
    else:
      doc.cats['positive'] = 0
      doc.cats['negative'] = 1

    #Adding the doc into the list 'text'
    text.append(doc)
  return(text)

In [ ]:
# Calculate the time for converting into binary document for train dataset
# Currently path is set as /content/drive/MyDrive/TrueFoundry/train.spacy 
# This is because I performed training on Google Collab.

# If running locally set path as train.spacy

start_time = datetime.now()

#passing the train dataset into function 'document'
train_docs = document(train)

#Creating binary document using DocBin function in spaCy
doc_bin = DocBin(docs = train_docs)

#Saving the binary document as train.spacy
doc_bin.to_disk("/content/drive/MyDrive/TrueFoundry/train.spacy")
end_time = datetime.now()

#Printing the time duration for train dataset
print('Duration: {}'.format(end_time - start_time))

Duration: 0:07:17.079788


In [ ]:
# Calculate the time for converting into binary document for test dataset
# Currently path is set as /content/drive/MyDrive/TrueFoundry/valid.spacy 
# This is because I performed training on Google Collab.

# If running locally set path as valid.spacy

start_time = datetime.now()

#passing the test dataset into function 'document'
test_docs = document(test)
doc_bin = DocBin(docs = test_docs)
doc_bin.to_disk("/content/drive/MyDrive/TrueFoundry/valid.spacy")
end_time = datetime.now()

#Printing the time duration for test dataset
print('Duration: {}'.format(end_time - start_time))

Duration: 0:01:37.888987


**6. Train the model**

In [ ]:
#Converting base configuration into full config file
# Currently path is set as /content/drive/MyDrive/TrueFoundry/base_config.cfg and /content/drive/MyDrive/TrueFoundry/config.cfg 
# This is because I performed training on Google Collab.

# If running locally set path as base_config.cfg and config.cfg respectively.

!python -m spacy init fill-config /content/drive/MyDrive/TrueFoundry/base_config.cfg /content/drive/MyDrive/TrueFoundry/config.cfg

✔ Auto-filled config with all values
✔ Saved config
/content/drive/MyDrive/TrueFoundry/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [4]:
#Calculating the time for training the model
# Currently path is set as /content/drive/MyDrive/TrueFoundry/config.cfg &
# /content/drive/MyDrive/TrueFoundry/output_updated because I performed training on Google Collab.

# If running locally set path as config.cfg and output_updated respectively.

start_time = datetime.now()
# To train the model. Enabled GPU and storing the model output in folder called output_updated
!python -m spacy train /content/drive/MyDrive/TrueFoundry/config.cfg --verbose --gpu-id 0 --output /content/drive/MyDrive/TrueFoundry/output_updated

end_time = datetime.now()
#Printing the time taken for training the model
print('Duration: {}'.format(end_time - start_time))

ℹ Saving to output directory:
/content/drive/MyDrive/TrueFoundry/output_updated
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2022-12-27 09:01:21,589] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-27 09:01:21,599] [DEBUG] Loading corpus from path: /content/drive/MyDrive/TrueFoundry/valid.spacy
DEBUG:spacy:Loading corpus from path: /content/drive/MyDrive/TrueFoundry/valid.spacy
[2022-12-27 09:01:21,600] [DEBUG] Loading corpus from path: /content/drive/MyDrive/TrueFoundry/train.spacy
DEBUG:spacy:Loading corpus from path: /content/drive/MyDrive/TrueFoundry/train.spacy
[2022-12-27 09:01:21,600] [INFO] Pipeline: ['transformer', 'textcat']
INFO:spacy:Pipeline: ['transformer', 'textcat']
[2022-12-27 09:01:21,603] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-27 09:01:21,604] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
Downloading: 100% 481/481 [0

**7. Testing the model**

In [5]:
# Loading the best model from output_updated folder
# Currently path is set as /content/drive/MyDrive/TrueFoundry/output_updated/model-best because I performed training on Google Collab.
# If running locally set path as output_updated/model-best.

nlp = spacy.load("/content/drive/MyDrive/TrueFoundry/output_updated/model-best")

In [7]:
text = "Australia’s largest airline temporarily lays off 2,500 employees"
demo = nlp(text)
print(demo.cats)

{'positive': 0.0026326205115765333, 'negative': 0.9973674416542053}


In [16]:
text = "@VirginAmerica plus you've added commercials to the experience... tacky."
demo = nlp(text)
print(demo.cats)
sent = demo.cats
max(sent, key=sent.get)

{'positive': 0.9922516942024231, 'negative': 0.007748330477625132}


'positive'

In [18]:
text = "flight was good. Descent staff!"
demo = nlp(text)
print(demo.cats)
sent = demo.cats
max(sent, key=sent.get)

{'positive': 0.9237127900123596, 'negative': 0.07628725469112396}


'positive'